In [1]:
import pandas as pd
from scripts.utils import filter_person_and_date
from scripts.MapMatch import MapMatch
from notebooks.scripts.Segment import Segment
from scripts.PlotMap import PlotMap
from scripts.KalmanFilter import kalman_filter

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
all_plt_data = pd.read_csv('../flask-app/static/data/all_plt_data.csv')

## Prepare GPS data -> json

In [5]:
person_df = filter_person_and_date(all_plt_data, 20, '2011-08-27')
print(person_df.head())

         person        lat        long  zero  altitude  date_numb_days  \
1144412      20  39.978668  116.307982     0       0.0    40782.399109   
1144413      20  39.978653  116.308022     0       0.0    40782.399120   
1144414      20  39.978640  116.308008     0       0.0    40782.399132   
1144415      20  39.978623  116.307967     0       0.0    40782.399144   
1144416      20  39.978628  116.307992     0       0.0    40782.399155   

               date      time               cst_datetime  cst_weekday  
1144412  2011-08-27  09:34:43  2011-08-27 17:34:43+08:00            5  
1144413  2011-08-27  09:34:44  2011-08-27 17:34:44+08:00            5  
1144414  2011-08-27  09:34:45  2011-08-27 17:34:45+08:00            5  
1144415  2011-08-27  09:34:46  2011-08-27 17:34:46+08:00            5  
1144416  2011-08-27  09:34:47  2011-08-27 17:34:47+08:00            5  


In [62]:
import pandas as pd
import requests
import json

def prepare_meili(person_df, colnames=['lat', 'long', 'cst_datetime']):
    """
    Prepare a person_df for map matching with Meili
    @param:
        - person_df: a pandas DataFrame containing the person's data
        - colnames: a list of the column names for latitude, longitude, and time
    @return:
        - request_body: a JSON string to be sent to the Meili API
    """
    lat_col, long_col, time_col = colnames
    person_df['time'] = pd.to_datetime(person_df[time_col]).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    prepared_df = person_df[[long_col, lat_col, 'time']].copy()
    prepared_df.columns = ['lon', 'lat', 'time']
    
    meili_coordinates = prepared_df.to_json(orient='records')
    request_body = f'{{"shape": {meili_coordinates}, "search_radius": 150, "shape_match":"map_snap", "costing":"auto", "format":"osrm"}}'
    return request_body


def meili_match(person_df, colnames=['lat', 'long', 'cst_datetime']):
    """
    Match a person's data to the road network using Meili
    @param:
        - person_df: a pandas DataFrame containing the person's data
        - colnames: a list of the column names for latitude, longitude, and time
    @return:
        - matched_df: a pandas DataFrame containing the matched data
    """
    request_body = prepare_meili(person_df, colnames)
    HEADERS= {'Content-Type': 'application/json'}
    URL = 'http://localhost:8002/trace_route'
    response = requests.post(URL, data=request_body, headers=HEADERS)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to match map: {response.status_code}\n{response.text}")

def parse_meili(meili_json, person_df):
    matchings = pd.json_normalize(meili_json, 'matchings')


meili_json = meili_match(person_df)

/var/folders/_1/wpkj3xlj2sxd981vsh_0fhd80000gn/T/ipykernel_31720/631613121.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  match_df = pd.concat([match_df, pd.DataFrame(rows)], ignore_index=True)


In [59]:
def make_matchdf(meili_json):
    """
    Create a dataframe of 'matchings' array from meili json response which includes 
    bearing / intersection / duration / distance / transportation type information
    @param:
        - meili_json: a json response from meili API
    @return:   
        - matching_df: pd.DataFrame containing matching information
    """
    matching_rows = []
    for matching_index, matching in enumerate(meili_json['matchings']):
        for leg_index, leg in enumerate(matching['legs']):

            # Extract via waypoints for the current leg
            via_waypoints = leg.get('via_waypoints', [])

            for step_index, step in enumerate(leg['steps']):
                # All step info
                step_row = {
                    'matching_index': matching_index,
                    # 'leg_index': leg_index,
                    'step_index': step['intersections'][0].get('geometry_index', 0),
                    'weight_name': matching.get('weight_name', ''),
                    'match_weight': matching.get('weight', 0),
                    'match_duration_sec': matching.get('duration', 0),
                    'match_distance': matching.get('distance', 0),
                    'leg_distance': leg.get('distance', 0),
                    'leg_duration': leg.get('duration', 0),
                    'leg_weight': leg.get('weight', 0),
                    'step_name': step.get('name', ''),
                    'step_duration': step.get('duration', 0),
                    'step_distance': step.get('distance', 0),
                    'step_weight': step.get('weight', 0),
                    'step_mode': step.get('mode', ''),
                    'driving_side': step.get('driving_side', ''),
                    'step_geometry': step.get('geometry', ''),
                    'instruction': step['maneuver'].get('instruction', ''),
                    'type': step['maneuver'].get('type', ''),
                    'bearing_after': step['maneuver'].get('bearing_after', 0),
                    'bearing_before': step['maneuver'].get('bearing_before', 0),
                    'maneuver_location': step['maneuver']['location']  # Assuming 'location' always exists in 'maneuver'
                }

                # Identify and include waypoint information that correlates to the current step
                for waypoint in via_waypoints:
                    if waypoint.get('geometry_index', -1) == step['intersections'][0].get('geometry_index', -2):
                        waypoint_index = waypoint.get('waypoint_index', None)
                        waypoint_distance_from_start = waypoint.get('distance_from_start', None)
                    else:
                        waypoint_index = None
                        waypoint_distance_from_start = None
                    step_row.update({
                        'waypoint_index': waypoint_index,
                        'waypoint_distance_from_start': waypoint_distance_from_start
                    })
                
                # Append the step_row information to matching_rows
                matching_rows.append(step_row)

    # Create a DataFrame from the list of rows
    matching_df = pd.DataFrame(matching_rows)
    return matching_df

matching_df = make_matchdf(meili_json)
matching_df

,matching_index,step_index,weight_name,match_weight,match_duration_sec,match_distance,leg_distance,leg_duration,leg_weight,step_name,...,step_mode,driving_side,step_geometry,instruction,type,bearing_after,bearing_before,maneuver_location,waypoint_index,waypoint_distance_from_start
0,0,0,auto,1071.938,147.760,882.673,882.673,147.760,1071.938,,...,driving,left,suwmkAw}sy|ExJtY,Drive southwest.,depart,240,0,"[116.304364, 40.087914]",NaN,NaN
1,0,1,auto,1071.938,147.760,882.673,882.673,147.760,1071.938,,...,driving,left,yiwmkAacsy|ERM,Turn left.,turn,152,240,"[116.303937, 40.087725]",256.0,42.000
2,0,2,auto,1071.938,147.760,882.673,882.673,147.760,1071.938,,...,driving,left,eiwmkAocsy|ESL,Make a right U-turn.,continue,332,152,"[116.303944, 40.087715]",NaN,NaN
3,0,3,auto,1071.938,147.760,882.673,882.673,147.760,1071.938,,...,driving,left,yiwmkAacsy|EyJuY,Turn right.,turn,60,332,"[116.303937, 40.087725]",NaN,NaN
4,0,4,auto,1071.938,147.760,882.673,882.673,147.760,1071.938,,...,driving,left,suwmkAw}sy|EsWbRcE{Kua@`Z,Turn left.,turn,329,60,"[116.304364, 40.087914]",NaN,NaN
5,0,7,auto,1071.938,147.760,882.673,882.673,147.760,1071.938,,...,driving,left,awymkAm|ry|EhB`G,Turn left.,turn,242,329,"[116.303831, 40.088961]",NaN,NaN
6,0,8,auto,1071.938,147.760,882.673,882.673,147.760,1071.938,,...,driving,left,wsymkAktry|Eya@zV}Ivk@_AxLqFtDoEhCiHtEjD~K,Turn right.,turn,332,242,"[116.303702, 40.088908]",NaN,NaN
7,0,15,auto,1071.938,147.760,882.673,882.673,147.760,1071.938,,...,driving,left,ou{mkAgdoy|EoPzJ,Turn right.,turn,333,242,"[116.301908, 40.08996]",NaN,NaN
8,0,16,auto,1071.938,147.760,882.673,882.673,147.760,1071.938,北农路,...,driving,left,_g|mkAkxny|Er\`aAno@riB,Turn left onto 北农路.,turn,240,333,"[116.301718, 40.09024]",NaN,NaN
9,0,18,auto,1071.938,147.760,882.673,882.673,147.760,1071.938,,...,driving,left,{xymkAukiy|EzIkItHoAjFnExNx`@,Turn left.,turn,144,239,"[116.298955, 40.08899]",NaN,NaN


In [70]:
def make_tracedf(meili_json, person_df):
    trace_rows = []
    for trace_index, tracepoint in enumerate(meili_json['tracepoints']):
        if tracepoint is None:
            trace_row = {
                'trace_index': trace_index,
                'matchings_index': None,
                'matched_lat': None,
                'matched_long': None,
                'alternatives_count': None,
                'trace_distance_from_start': None,
                'trace_name': None,
                'trace_waypoint_index': None,
                'cst_datetime': person_df.iloc[trace_index]['cst_datetime']
            }
        else:
            trace_row = {
                'trace_index': trace_index,
                'matchings_index': tracepoint.get('matchings_index', None),
                'matched_lat': tracepoint.get('location', [None, None])[1],
                'matched_long': tracepoint.get('location', [None, None])[0],
                'alternatives_count': tracepoint.get('alternatives_count', 0),
                'trace_distance_from_start': tracepoint.get('distance_from_start', 0),
                'trace_name': tracepoint.get('name', ''),
                'waypoint_index': tracepoint.get('waypoint_index', None),
                'cst_datetime': person_df.iloc[trace_index]['cst_datetime']
            }
        trace_rows.append(trace_row)

    trace_df = pd.DataFrame(trace_rows)
    return trace_df

trace_df = make_tracedf(meili_json, person_df)
trace_df

,trace_index,matchings_index,matched_lat,matched_long,alternatives_count,trace_distance_from_start,trace_name,waypoint_index,cst_datetime,trace_waypoint_index
0,0,0.0,40.087914,116.304364,0.0,0.0,,0.0,2008-11-09 09:45:30+08:00,NaN
1,1,0.0,40.087870,116.304265,0.0,0.0,,NaN,2008-11-09 09:45:32+08:00,NaN
2,2,0.0,40.087843,116.304205,1.0,0.0,,NaN,2008-11-09 09:45:34+08:00,NaN
3,3,0.0,40.087829,116.304172,0.0,0.0,,NaN,2008-11-09 09:45:36+08:00,NaN
4,4,0.0,40.087828,116.304169,0.0,0.0,,NaN,2008-11-09 09:45:38+08:00,NaN
...,...,...,...,...,...,...,...,...,...,...
454,454,9.0,40.050193,116.300105,0.0,0.0,G7,NaN,2008-11-09 10:28:08+08:00,NaN
455,455,9.0,40.050147,116.300136,0.0,0.0,G7,NaN,2008-11-09 10:28:10+08:00,NaN
456,456,9.0,40.050100,116.300167,0.0,0.0,G7,NaN,2008-11-09 10:28:12+08:00,NaN
457,457,9.0,40.050052,116.300199,0.0,0.0,G7,NaN,2008-11-09 10:28:14+08:00,NaN


In [29]:
import json

with open('meili_json.json', 'w') as file:
    json.dump(meili_json, file, indent=2)


In [25]:
person_df

,person,lat,long,zero,altitude,date_numb_days,date,time,cst_datetime,cst_weekday
1144412,20,39.978668,116.307982,0,0.0,40782.399109,2011-08-27,2011-08-27T17:34:43Z,2011-08-27T17:34:43+0800,5
1144413,20,39.978653,116.308022,0,0.0,40782.399120,2011-08-27,2011-08-27T17:34:44Z,2011-08-27T17:34:44+0800,5
1144414,20,39.978640,116.308008,0,0.0,40782.399132,2011-08-27,2011-08-27T17:34:45Z,2011-08-27T17:34:45+0800,5
1144415,20,39.978623,116.307967,0,0.0,40782.399144,2011-08-27,2011-08-27T17:34:46Z,2011-08-27T17:34:46+0800,5
1144416,20,39.978628,116.307992,0,0.0,40782.399155,2011-08-27,2011-08-27T17:34:47Z,2011-08-27T17:34:47+0800,5
...,...,...,...,...,...,...,...,...,...,...
1165825,20,39.978552,116.304558,0,0.0,40782.334410,2011-08-27,2011-08-27T16:01:33Z,2011-08-27T16:01:33+0800,5
1165826,20,39.978548,116.304568,0,0.0,40782.334421,2011-08-27,2011-08-27T16:01:34Z,2011-08-27T16:01:34+0800,5
1165827,20,39.978545,116.304572,0,0.0,40782.334433,2011-08-27,2011-08-27T16:01:35Z,2011-08-27T16:01:35+0800,5
1165828,20,39.978543,116.304567,0,0.0,40782.334444,2011-08-27,2011-08-27T16:01:36Z,2011-08-27T16:01:36+0800,5


In [61]:
match_df

,name,distance,roadsnap_long,roadsnap_lat,batch_index,matchings_index,confidence,via_waypoints,admins,weight,duration,steps,summary,matching_index


In [75]:
# match_df = match_df.dropna()
# match_df['batch_index'] = -1
# match_df['matchings_index'] = -1
# match_df['confidence'] = 1

pm = PlotMap(person_df, tile_type='light')
pm.full_polyline(person_df, 'original')
pm.full_polyline(trace_df, 'matched')

pm.circles(person_df, 'original')
pm.circles(trace_df, 'matched')
pm.show()

## Send requests to Valhalla Meili service

In [33]:
import json

person_df = filter_person_and_date(all_plt_data, 179, '2008-11-09')

meili_json = meili_match(person_df)
match_df = make_matchdf(meili_json)

with open('meili_json.json', 'w') as file:
    json.dump(meili_json, file, indent=2)

In [32]:
match_df = match_df.dropna()
match_df['batch_index'] = -1
match_df['matchings_index'] = -1
match_df['confidence'] = 1

pm = PlotMap(person_df)
pm.full_polyline(person_df, 'original')
pm.full_polyline(match_df, 'road_snapped')

pm.circles(person_df, 'original')
pm.circles(match_df, 'road_snapped')
pm.show()